## Upload CSV Files

In [65]:
import pandas as pd
import sys
from collections import defaultdict

data_dir = '/home/john/enviro_papers/data/'

rep_and_district_info_filename = 'Names_Districts_Counties.csv'
county_asthma_info_filename = 'Asthma_Data_ALA_6.26.2019.csv'
county_polling_info_filename = 'Yale_Polling.csv'
voting_history_filename = 'vote_history.csv'

In [66]:
rep_and_district_info = pd.read_csv(data_dir + rep_and_district_info_filename, encoding = "ISO-8859-1")
county_asthma_info = pd.read_csv(data_dir + county_asthma_info_filename, encoding = "ISO-8859-1")
county_polling_info = pd.read_csv(data_dir + county_polling_info_filename, encoding = "ISO-8859-1")
voting_history = pd.read_csv(data_dir + voting_history_filename, encoding = "ISO-8859-1")

In [67]:
# Make sure the data was read correctly and is what we expect

print(rep_and_district_info.head(3))
print(county_asthma_info.head(3))
print(county_polling_info.head(3))
print(voting_history.head(3))

   Branch First Name Last Name District Party  \
0  Senate     George    Barker       39     D   
1  Senate    Richard     Black       13     R   
2  Senate   Jennifer    Boysko       33     D   

                                            Counties  
0  Fairfax County, Prince William County, Alexand...  
1            Loudoun County, Prince William County,   
2                   Fairfax County, Loudoun County,   
            County Total Pop Under 18 65 & Over Pediatric Asthma Adult Asthma  \
0         Accomack    32,545    6,756     7,402              536        2,245   
1        Albemarle   107,702   21,636    19,153            1,719        7,565   
2  Alexandria City   160,035   28,866    17,715            2,293       11,685   

    COPD Lung Cancer CV Disease Diabetes Poverty Estimate  
0  2,045          18      2,723    3,222            5,715  
1  6,096          59      7,726    9,287            8,015  
2  8,058          88      9,353   11,706           15,922  
  GeoType    GeoNa

## Clean County Lists


In [68]:
rep_and_district_info_counties = rep_and_district_info['Counties']
rep_and_district_info_districts = rep_and_district_info['District']

#Clean up district values
rep_and_district_info_counties = pd.Series(rep_and_district_info_counties.str.split(pat=","))

all_counties = []
county_lists = []
for district_to_counties_list in rep_and_district_info_counties:
    sanitized_counties = []
    for county in district_to_counties_list:
        sanitized_county_name = county.replace(' County','').strip()
        if sanitized_county_name:
            all_counties.append(sanitized_county_name)
            sanitized_counties.append(sanitized_county_name)
    county_lists.append(sanitized_counties)    
            
districts = list(rep_and_district_info['District'])
for i in xrange(len(districts)):        
    if "th" in districts[i] or "st" in districts[i] or "nd" in districts[i] or "rd" in districts[i]:
            districts[i] = districts[i][:-2]
            
district_to_counties = dict(zip(districts, county_lists))

all_counties = list(set(all_counties))
all_counties = list([i for i in all_counties if i])

In [69]:
# Sanity check the outputs

district_to_counties

{u'1': [u'Lee', u'Scott', u'Wise'],
 u'10': [u'Clarke', u'Frederick', u'Loudoun'],
 u'100': [u'Accomack', u'Northampton'],
 u'11': [u'Roanoke City'],
 u'12': [u'Giles', u'Montgomery', u'Pulaski'],
 u'13': [u'Prince William'],
 u'14': [u'Henry', u'Pittsylvania'],
 u'15': [u'Page', u'Rockingham', u'Shenandoah', u'Warren'],
 u'16': [u'Henry', u'Pittsylvania'],
 u'17': [u'Botetourt', u'Roanoke'],
 u'18': [u'Culpeper', u'Fauquier', u'Rappahannock', u'Warren'],
 u'19': [u'Alleghany', u'Bedford', u'Botetourt'],
 u'2': [u'Prince William', u'Stafford'],
 u'20': [u'Augusta', u'Highland', u'Nelson'],
 u'21': [u'Chesapeake City', u'Virginia Beach City'],
 u'22': [u'Bedford', u'Campbell', u'Franklin'],
 u'23': [u'Amherst', u'Bedford'],
 u'24': [u'Amherst', u'Augusta', u'Bath', u'Rockbridge'],
 u'25': [u'Albemarle', u'Augusta', u'Rockingham'],
 u'26': [u'Rockingham'],
 u'27': [u'Chesterfield'],
 u'28': [u'Stafford'],
 u'29': [u'Frederick', u'Warren'],
 u'3': [u'Bland', u'Buchanan', u'Russell', u'Taz

In [70]:
# Drop labels 133 to get rid of "total" column
asthma_info_counties = list(county_asthma_info['County'].drop(labels=133))
asthma_info_children = list(county_asthma_info['Pediatric Asthma'].drop(labels=133))
asthma_info_adults = list(county_asthma_info['Adult Asthma'].drop(labels=133))

asthma_info_children = [int(x.replace(',', '')) for x in asthma_info_children]
asthma_info_adults = [int(x.replace(',', '')) for x in asthma_info_adults]

county_to_asthma_children = dict(zip(asthma_info_counties, asthma_info_children))
county_to_asthma_adults = dict(zip(asthma_info_counties, asthma_info_adults))

In [71]:
county_to_asthma_adults

{u'Accomack': 2245,
 u'Albemarle': 7565,
 u'Alexandria City': 11685,
 u'Alleghany': 1064,
 u'Amelia': 904,
 u'Amherst': 2209,
 u'Appomattox': 1076,
 u'Arlington': 17210,
 u'Augusta': 5311,
 u'Bath': 315,
 u'Bedford': 5461,
 u'Bland': 465,
 u'Botetourt': 2349,
 u'Bristol City': 1167,
 u'Brunswick': 1174,
 u'Buchanan': 1549,
 u'Buckingham': 1230,
 u'Buena Vista City': 450,
 u'Campbell': 3871,
 u'Caroline': 2056,
 u'Carroll': 2104,
 u'Charles City': 516,
 u'Charlotte': 830,
 u'Charlottesville City': 3592,
 u'Chesapeake City': 16141,
 u'Chesterfield': 23149,
 u'Clarke': 1013,
 u'Colonial Heights City': 1184,
 u'Covington City': 393,
 u'Craig': 360,
 u'Culpeper': 3391,
 u'Cumberland': 686,
 u'Danville City': 2799,
 u'Dickenson': 1032,
 u'Dinwiddie': 1987,
 u'Emporia City': 355,
 u'Essex': 775,
 u'Fairfax': 78010,
 u'Fairfax City': 1622,
 u'Falls Church City': 968,
 u'Fauquier': 4695,
 u'Floyd': 1097,
 u'Fluvanna': 1851,
 u'Franklin': 3967,
 u'Franklin City': 544,
 u'Frederick': 5844,
 u'Fre

# Asthma

## Sum childhood, adult, and total asthma for the counties that make up each district

Text: In the four counties that make up House District 100, 10,000 kids and 20,000 adults live with asthma.

In [72]:
# Make a mapping of istrict to asthma totals, # of counts of district.

district_to_asthma_text = {}
for district,counties in district_to_counties.iteritems():    
    child_asthma_count = 0
    adult_asthma_count = 0
    for county in counties:
        child_asthma_count += county_to_asthma_children[county]
        adult_asthma_count += county_to_asthma_adults[county]
    
    if len(counties) > 1:
        district_to_asthma_text[district] =  """"In the %d counties that make up House District %s, %d kids and %d adults live with asthma.""" % (len(counties), district, child_asthma_count, adult_asthma_count)
    else:
        district_to_asthma_text[district] =  """In the House District %s, %d kids and %d adults live with asthma.""" % (district, child_asthma_count, adult_asthma_count)

In [73]:
district_to_asthma_text

{u'1': u'"In the 3 counties that make up House District 1, 1272 kids and 5974 adults live with asthma.',
 u'10': u'"In the 3 counties that make up House District 10, 10832 kids and 32306 adults live with asthma.',
 u'100': u'"In the 2 counties that make up House District 100, 721 kids and 3068 adults live with asthma.',
 u'11': u'In the House District 11, 1784 kids and 6820 adults live with asthma.',
 u'12': u'"In the 3 counties that make up House District 12, 1982 kids and 11006 adults live with asthma.',
 u'13': u'In the House District 13, 10040 kids and 30053 adults live with asthma.',
 u'14': u'"In the 2 counties that make up House District 14, 1729 kids and 7908 adults live with asthma.',
 u'15': u'"In the 4 counties that make up House District 15, 3203 kids and 12842 adults live with asthma.',
 u'16': u'"In the 2 counties that make up House District 16, 1729 kids and 7908 adults live with asthma.',
 u'17': u'"In the 2 counties that make up House District 17, 2000 kids and 8892 ad

# Polling

## Create a weighted average of polling results for the counties that make up each district

Text: 

In the counties that make up this district, [weighted average]% know that global warming is happening, and [weighted average] are somewhat or very worried about it. [weighted average]% support regulating CO2 as a pollutant, and [weighted average] want to provide tax rebates for people who purchase energy-efficient vehicles or solar panels. 


- Number of people who know that global warming is happening: 
    - [[x%]] in [[least populous county]], and [[y%]] in [[most populous county]]
- Percent who are somewhat or very worried about climate change: 
    - [[x%]] in [[least populous county]], and [[y%]] in [[most populous county]]
- Support regulating CO2 as a pollutant: 
    - [[x%]] in [[least populous county]], and [[y%]] in [[most populous county]]
- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: 
    - [[x%]] in [[least populous county]], and [[y%]] in [[most populous county]]


In [74]:
county_polling_info_counties = list(county_polling_info['GeoName'])
county_polling_info_pop = dict(zip(county_polling_info_counties,list(county_polling_info['TotalPop'])))
county_polling_info_happening = dict(zip(county_polling_info_counties,list(county_polling_info['happening'])))
county_polling_info_worried = dict(zip(county_polling_info_counties,list(county_polling_info['worried'])))
county_polling_info_regulate = dict(zip(county_polling_info_counties,list(county_polling_info['regulate'])))
county_polling_info_rebates = dict(zip(county_polling_info_counties,list(county_polling_info['rebates'])))

In [75]:
district_to_overview = {}
for district,counties in district_to_counties.iteritems():    
    total_pop, know_num, worried_num, regulate_num, rebates_num = 0.0,0.0,0.0,0.0,0.0
    
    for county in counties:
        total_pop += county_polling_info_pop[county]
        know_num += county_polling_info_happening[county] * county_polling_info_pop[county]
        worried_num += county_polling_info_worried[county] * county_polling_info_pop[county]
        regulate_num += county_polling_info_regulate[county] * county_polling_info_pop[county]
        rebates_num += county_polling_info_rebates[county] * county_polling_info_pop[county]
        
    district_to_overview[district] =  ("In the counties that make up"
                                       " this district, {0:.2f}% know that"
                                       " global warming is happening,"
                                       " and {1:.2f}% are somewhat or very"
                                       " worried about it. {2:.2f}% support"
                                       " regulating CO2 as a pollutant,"
                                       " and {3:.2f}% want to provide tax"
                                       " rebates for people who purchase"
                                       " energy-efficient vehicles or "
                                       "solar panels.").format(know_num/total_pop, worried_num/total_pop, regulate_num/total_pop, rebates_num/total_pop)

In [76]:
print(district_to_overview)

{u'24': 'In the counties that make up this district, 60.96% know that global warming is happening, and 50.17% are somewhat or very worried about it. 72.59% support regulating CO2 as a pollutant, and 78.38% want to provide tax rebates for people who purchase energy-efficient vehicles or solar panels.', u'25': 'In the counties that make up this district, 66.92% know that global warming is happening, and 55.76% are somewhat or very worried about it. 75.62% support regulating CO2 as a pollutant, and 80.88% want to provide tax rebates for people who purchase energy-efficient vehicles or solar panels.', u'26': 'In the counties that make up this district, 61.67% know that global warming is happening, and 50.18% are somewhat or very worried about it. 73.22% support regulating CO2 as a pollutant, and 78.30% want to provide tax rebates for people who purchase energy-efficient vehicles or solar panels.', u'27': 'In the counties that make up this district, 69.64% know that global warming is happen

In [77]:
district_to_specific = {}
for district,counties in district_to_counties.iteritems():
    max_pop_county, min_pop_county = '',''
    max_pop = -1
    min_pop = sys.maxint

    
    for county in counties:
        if county_polling_info_pop[county] > max_pop:
            max_pop = county_polling_info_pop[county]
            max_pop_county = county
        if county_polling_info_pop[county] < min_pop:
            min_pop = county_polling_info_pop[county]
            min_pop_county = county                        
    
    if not max_pop_county or not min_pop_county:
        continue
    min_pop_happening = county_polling_info_happening[min_pop_county]
    min_pop_worried = county_polling_info_worried[min_pop_county]
    min_pop_regulate = county_polling_info_regulate[min_pop_county]
    min_pop_rebates = county_polling_info_rebates[min_pop_county]
    max_pop_happening = county_polling_info_happening[max_pop_county]
    max_pop_worried = county_polling_info_worried[max_pop_county]
    max_pop_regulate = county_polling_info_regulate[max_pop_county]
    max_pop_rebates = county_polling_info_rebates[max_pop_county]
    
    district_to_specific[district] = ("- Number of people who know that global warming is happening:" +
        "\n\t- {0:.2f}% in {1}, and {2:.2f}% in {3}").format(min_pop_happening, min_pop_county, max_pop_happening, max_pop_county)
    district_to_specific[district] += (
    "\n- Percent who are somewhat or very worried about climate change: "
        "\n\t- {0:.2f}% in {1}, and {2:.2f}% in {3}").format(min_pop_worried, min_pop_county, max_pop_worried, max_pop_county)

    district_to_specific[district] += (
    "\n- Support regulating CO2 as a pollutant: "
        "\n\t- {0:.2f}% in {1}, and {2:.2f}% in {3}"
    ).format(min_pop_regulate, min_pop_county, max_pop_regulate, max_pop_county)

    district_to_specific[district] += (
    "\n- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: "
        "\n\t- {0:.2f}% in {1}, and {2:.2f}% in {3}").format(min_pop_rebates, min_pop_county, max_pop_rebates, max_pop_county)

In [78]:
print(district_to_specific)

{u'24': '- Number of people who know that global warming is happening:\n\t- 59.87% in Bath, and 59.15% in Augusta\n- Percent who are somewhat or very worried about climate change: \n\t- 48.84% in Bath, and 48.67% in Augusta\n- Support regulating CO2 as a pollutant: \n\t- 72.42% in Bath, and 71.72% in Augusta\n- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: \n\t- 78.39% in Bath, and 77.75% in Augusta', u'25': '- Number of people who know that global warming is happening:\n\t- 61.67% in Rockingham, and 76.88% in Albemarle\n- Percent who are somewhat or very worried about climate change: \n\t- 50.18% in Rockingham, and 65.45% in Albemarle\n- Support regulating CO2 as a pollutant: \n\t- 73.22% in Rockingham, and 80.45% in Albemarle\n- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: \n\t- 78.30% in Rockingham, and 85.23% in Albemarle', u'26': '- Number of people who know that global warming is happening:\n\t- 61

# Opposition Research

1. Build out an excel file with the following:
   - Name of incumbent
   - Important bills, and a vote (positive/negative)
   - 2 descriptions for each bill: positive and negative
   - The correct description to use, based on the vote history

2. Match incumbent names into the correct description, then add all descriptions together to create an opposition research paragraph that looks like this:

When it comes to common sense clean energy reforms, [[incumbent]] has a mixed record. They voted to block Virginia from entering regional collaborative efforts to combat climate change, specifically by voting to prevent Virginia from entering the Regional Greenhouse Gas Initiative. States already in this program have seen lower electricity bills and less pollution – joining it is a no-brainer. [[He/She]] also voted against integrating environmental education into Virginia's classrooms. 

However, [[incumbent]]’s record is not all bad. [[incumbent last name]] voted to ensure that the State Corporation Commission cannot reject or cut climate-forward legislation without reason. They also voted to establish a Clean Energy Advisory Board, which will run a pilot program to allow low-to-moderate income households to receive rebates for solar panels. Finally, [[Incumbent]] voted to ensure that coal plants dispose of coal ash– one of the largest types of industrial waste generated in the United States– safely and sustainably.



In [79]:
rep_and_district_info_districts = list(rep_and_district_info['District'])
rep_and_district_info_candidate_first = list(rep_and_district_info['First Name'])
rep_and_district_info_candidate_last = list(rep_and_district_info['Last Name'])

sanitized_candidate_names = []
for candidate_name in zip(rep_and_district_info_candidate_first,rep_and_district_info_candidate_last):
    sanitized_candidate_names.append(candidate_name[0] + " " + candidate_name[1])
    
candidate_to_district = {}
for candidate_district_pair in zip(sanitized_candidate_names, rep_and_district_info_districts):
    candidate = candidate_district_pair[0]
    district = candidate_district_pair[1]
    if "th" in district or "st" in district or "nd" in district or "rd" in district:
        candidate_to_district[candidate] = district[:-2]
    else:
        candidate_to_district[candidate] = district

In [80]:
candidate_to_voting_record_sentences = defaultdict(list)

for row in voting_history.as_matrix():
    candidate_name = row[1] + " " + row[2]
    for i in xrange(4, len(row)):
        candidate_to_voting_record_sentences[candidate_name].append(row[i].replace("Candidate", candidate_name))            

/home/john/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [81]:
candidate_to_voting_record_sentences

defaultdict(list,
            {u'Adam Ebbin': [u'-',
              u'-',
              u'-',
              u'-',
              u'Adam Ebbin voted to make it more difficult for Virginia to enter regional collaborative efforts to combat climate change in the transportation sector (via the Transportation and Climate Initiative).',
              u'-',
              u'-',
              u'Adam Ebbin voted to make it more difficult for Virginia to enter into a regional cap and trade program (RGGI). States in this program have seen lower electricity bills and more clean job growth - joining it makes common sense.',
              u'-',
              u'-',
              u'Adam Ebbin voted to ensure that coal plants dispose of coal ash- one of the largest types of industrial waste generated in the United States-\xc2\xa0safely and sustainably.',
              u'-',
              u"Adam Ebbin voted to preserve offshore habitat and limit fossil fuel dependency by prohibiting drilling in Virginia's w

In [82]:
# Parse the voting record
    # We have positive and negative votes, but I think we really just need the sentences
    # Drop first x columns
candidate_to_voting_record_text = {}
for candidate,voting_record_sentences in candidate_to_voting_record_sentences.iteritems():
    voting_record_text = ""
    for sentence in voting_record_sentences:
        if sentence != "-":
            voting_record_text += sentence + " "
    candidate_to_voting_record_text[candidate] = voting_record_text  

# Final Printing

In [83]:

for candidate in sanitized_candidate_names:
    d = candidate_to_district[candidate]
    print "________________________________________________________________________________________________"
    district_text = district_to_asthma_text[d] + "\n\n" + district_to_overview[d] + "\n\n" + district_to_specific[d] + "\n\n" + candidate_to_voting_record_text[candidate]
    
    print district_text
    print "________________________________________________________________________________________________"

________________________________________________________________________________________________
In the House District 39, 21376 kids and 78010 adults live with asthma.

In the counties that make up this district, 77.25% know that global warming is happening, and 69.92% are somewhat or very worried about it. 78.79% support regulating CO2 as a pollutant, and 84.10% want to provide tax rebates for people who purchase energy-efficient vehicles or solar panels.

- Number of people who know that global warming is happening:
	- 77.25% in Fairfax, and 77.25% in Fairfax
- Percent who are somewhat or very worried about climate change: 
	- 69.92% in Fairfax, and 69.92% in Fairfax
- Support regulating CO2 as a pollutant: 
	- 78.79% in Fairfax, and 78.79% in Fairfax
- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: 
	- 84.10% in Fairfax, and 84.10% in Fairfax

George Barker voted to make it more difficult for Virginia to enter regional collaborative efforts t